In [ ]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dropout, Dense
seed = 7
numpy.random.seed(seed)


# load pima indians dataset
# dataset = numpy.loadtxt(path, delimiter=",")
dataset = numpy.random.rand(800, 9)

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,  7]

# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, Y, nb_epoch=10, batch_size=10, verbose=0)

# evaluate the model
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))